# Example notebook for JWST MIRI imaging data reduction

Stacey Alberts <br>
Dec 8, 2023
<br><br>
This notebook includes examples of how to run the JWST calibration pipeline for MIRI imaging in python including stages 1-3 and background subtraction.  It is not meant to run a real example. 
<br><br>
Each stage of the pipeline has more customizable options than are shown here.  
Pipeline Documentation: https://jwst-pipeline.readthedocs.io/en/latest/index.html# <br>
More notebook examples: https://spacetelescope.github.io/jdat_notebooks/

In [ ]:
# general imports

import glob, os
import numpy as np

from jwst.pipeline import calwebb_detector1, calwebb_image2, calwebb_image3

from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base


In [ ]:
# Set CRDS paths if not set already in your .bashrc shell configuration
#os.environ['CRDS_PATH']='/Users/name/crds_cache'
#os.environ['CRDS_SERVER_URL']='https://jwst-crds.stsci.edu'
# Echo CRDS path in use
print('CRDS local filepath:',os.environ['CRDS_PATH'])
print('CRDS file server:',os.environ['CRDS_SERVER_URL'])

# Stage 1 - calwebb_detector1

In [ ]:
def miri_detector1(
    miri_uncal_files,
    output_dir,
    darkfile=None,
    linfile=None,
    save_jump_info=False,
    cpufraction="half",
):
    """
    Run CALWEBB_DETECTOR1 pipeline on uncal files.
    """
    for miri_uncal_file in miri_uncal_files:

        # Using the run() method:
        # Instantiate the pipeline
        miri1 = calwebb_detector1.Detector1Pipeline()

        # Save the final output of the pipeline
        miri1.output_dir = output_dir
        miri1.save_results = True

        # Turn on the cosmic ray shower correction
        miri1.jump.find_showers = True

        # the default detector correction should be used
        # but if you really wanted to override for example
        # the linearity or dark current files, you would
        # do it like this
        if linfile is not None:
            miri1.linearity.override_linearity = linfile

        if darkfile is not None:
            miri1.dark_current.override_dark = darkfile

        # set how much of your CPU the jump step can use
        miri1.jump.maximum_cores = cpufraction

        # Save the results of the ramp fitting
        # you can save intermedian result. This option saves 
        # the jump step files
        # https://jwst-pipeline.readthedocs.io/en/latest/jwst/jump/index.html
        if save_jump_info:
            miri1.save_calibrated_ramp = True
            miri1.ramp_fit.save_results = True
            miri1.ramp_fit.save_opt = True
                
        # set how much of your CPU the ramp fitting step can use
        miri1.ramp_fit.maximum_cores = cpufraction

        # Call the run() method
        miri1.run(miri_uncal_file)

In [ ]:
input_dir_stage0 = './stage0/' # path to uncal data
output_dir_stage1 = './stage1/' # path to output directory for stage 1

miri_uncal_files = glob.glob(f"{input_dir_stage0}*uncal.fits")

miri_detector1(miri_uncal_files, output_dir_stage1, linfile=None, 
                darkfile=None)

# Stage 2 - calwebb_image2

In [ ]:
def miri_image2(miri_rate_files, output_dir,
                useflat=True,
                flatfile=None,
                photomfile=None):
    """
    Run CALWEBB_IMAGE2 pipeline on rate files.
    """
    
    # Using the run() method
    # Instantiate the pipeline
    miri_im2 = calwebb_image2.Image2Pipeline()

    # Save the pipeline output, and specify the output directory
    miri_im2.output_dir = output_dir
    miri_im2.save_results = True

    # the default detector correction should be used
    # but if you really wanted to override for example
    # the flatfield or photom (flux calibration) files, you would
    # do it like this
    if flatfile is not None:
        miri_im2.flat_field.override_flat = flatfile
    if not useflat:
        miri_im2.flat_field.skip=True

    if photomfile is not None:
        miri_im2.photom.override_photom = photomfile

    # Call the run() method
    miri_im2.run(miri_rate_files)

In [ ]:
miri_rate_files = glob.glob(f"{output_dir_stage1}*_rate.fits")
output_dir_stage2 = './stage2/' # path to output directory for stage 2

miri_image2(miri_rate_files, output_dir_stage2, useflat=True)

# Background Subtraction

This is an outline of the general steps I use (adapted from the bkg subtraction method developed by Pablo Perez Gonzalez).  Your ideal bkg subtraction may depend on your dataset.

### Construction of a super background and subtraction from stage 2 cal files (Alvarez-Marquez+23, Lyu+23)

Note: if you have multiple pointings, use all dithers that were taken at roughly the same time (within a month or so) to construct the super backgrounds. If you only have closely space dithers of a single image, you may find you are oversubtracting around bright sources and need to modify the technique.
<br><br>


Step 1: construct a really good source mask
* Run the pipeline all the way through with no bkg subtraction to create an initial segmentation mask (seg_0)
* mask each cal file with seg_0 and filter the cal images (e.g. remove "jailbars" by subtracting the median in rows and columns, subtract a large-scale 2D gradient). Run the pipeline on these filtered cal files to create a new segmentation mask (seg_1). Optionally dilate the segments to better cover faint extended emission. 
* repeat the previous step one or more times until you have a segmentation mask you're happy with (seg_final).

Step 2: zero out the median background of each cal file
* Masking with seg_final, calculate the median background of each UNFILTERED cal file and subtract it.

Step 3: create a super background for each cal file
* Masking with seg_final, calculate the median background of a given UNFILTERED cal file.
* Construct the super background from the pixel-wise median of a stack of the ZERO-MEDIAN cal files (minus the cal file being corrected) and scale to the median of the cal file being corrected.
* Subtract the super brackground from the cal file in question.  Filter as desired (e.g. remove jailbars, large-scale gradients). 

Step 4: Repeat Step 3 for all cal files and run bkg subtracted cal files through stage 3.

# A Word on Astrometry Corrections

Stage 3 of the pipeline will run tweakreg to do an astrometry correction either using a GAIA catalog or a custom catalog you input.  A custom catalog option was not available for early versions of the pipeline, so we used external routines and I haven't tested the pipeline version, but I include some (untested) instructions for this.
<br><br>
HOWEVER, I don't like how the pipeline does it, since it does it at the Stage 3 mosaicking step and I found that correcting individual cal files first before mosaicking got better results.
<br><br>
On catalog, I've found that using GAIA generally doesn't work well for MIRI (not many matches with high SNR).  Long wavelength NIRCam imaging corrected to GAIA is ideal, but use whatever band with good astrometry that is closest to your MIRI filter.  If you have multiple MIRI filters, I found it's good to correct the shortest filter and then march it up (i.e. correct F770W to a F444W catalog, then correct F1000W to a the corrected F770W catalog).  I find that using as many matches as possible does as well or better than trying to pick compact sources.

# Stage 3 - calwebb_image3

Stage 3 takes association files which specify all of the relevant files to be combined.  The pipeline can generate this association file for you, but you can also construct your own if you do any external steps like background subtraction.

In [ ]:
def miri_image3(miri_asn_file,output_dir,
                output_file=None,
                tweakreg=False, abs_refcat=None, 
                use_custom_cat=False, custom_cat=None, 
                minobj=10., snr_threshold=5.0,
                crval=None, crpix=None,
                rotation=0,
                fillval = 'nan',
                output_shape=None,
                pixel_scale=None, use_skymatch=False):

    """
    Run CALWEBB_IMAGE3 pipeline on asn file.
    """
    # Create an instance of the pipeline class
    im3 = calwebb_image3.Image3Pipeline()
    im3.resample.save_results = True

    # set output directory
    im3.output_dir = output_dir
    if output_file:
        im3.output_file = output_file

    #=====================
    # astrometry correction options
    # https://jwst-pipeline.readthedocs.io/en/latest/jwst/tweakreg/README.html

    if not tweakreg:
        im3.tweakreg.skip = True
    else:
        # use a GAIA catalog? GAIADR1, GAIADR2, GAIADR3
        if abs_refcat is not None:
            im3.tweakreg.abs_refcat = abs_refcat

        # use custom catalog?
        if use_custom_cat:
            im3.tweakreg.use_custom_catalogs = True
            im3.tweakreg.snr_threshold = snr_threshold
            # minimum number of matched sources for correction
            im3.tweakreg.min_objs = minobj 

    # set if you don't want the pipeline to generate a catalog
    im3.source_catalog.skip = False
    
    #=====================
    # set resample options

    # RA and Dec of the reference pixel
    if crval is not None:
        im3.resample.crval = crval
    # position of reference pixel
    if crpix is not None:
        im3.resample.crpix = crpix

    # set the pixel scale (native = 0.11")
    if pixel_scale is not None:
        im3.resample.pixel_scale = pixel_scale

    # value for pixels with no data
    # default is 0
    if fillval is not None:
        im3.resample.fillval = fillval 

    # rotatio = 0 puts north up and east right
    if rotation is not None:
        im3.resample.rotation = rotation

    # specify the output shape of the final image or mosaic
    # useful if you have multiple filters and you want
    # everything the same shape
    if output_shape is not None:
        im3.resample.output_shape = output_shape
    
    #=====================

    # skymatch will attempt to match the sky levels between different pointings
    # using overlapping regions
    # I don't find it helps if you've already done background subtraction
    if not use_skymatch:
        im3.skymatch.skip = True
    # set the skymatch method
    # https://jwst-pipeline.readthedocs.io/en/latest/jwst/skymatch/description.html
    im3.skymatch.skymethod = 'match'

    # Call the run() method
    im3.run(miri_asn_file)

In [ ]:
# input is cal files directly from the pipeline or modified cal files
# if you do background subtraction
miri_cal_files = glob.glob(f"{output_dir_stage2}*_cal.fits")
output_dir_stage3 = './stage3/' # path to output directory for stage 3

# ==================
# create an asn file

# you can name it whatever you want, this is what I do
pid = '0000'
filter = 'F560W'
version = 1.0
miri_asn_name = f'jw{pid}_miri_{filter}_mosaic_v{version}'

miri_asn = asn_from_list.asn_from_list(miri_cal_files, rule=DMS_Level3_Base, product_name=miri_asn_name)
miri_asn_file = f'{miri_asn_name}.json'
with open(miri_asn_file, 'w') as outfile:
        name, serialized = miri_asn.dump(format='json')
        outfile.write(serialized)

# if you want to use a custom catalog for astrometry correction, you need
# to add it to the asn file.  I have no idea how to do this.

# name for final output mosaic
output_name = miri_asn_name

miri_image3(miri_asn_file, output_dir_stage3, output_file=output_name,
                crval=None, crpix=None, rotation=0, fillval='nan',
                pixel_scale=0.11, use_skymatch=False)